In [19]:
import pandas as pd
import os
import re

In [20]:
def rdmsp(lines):
    block = []
    for line in lines:
        if len(line)==1:
            yield block
            block = []
        else:
            block.append(line)
atom_mass = {'H': 1.00782503223,
             'C': 12,
             'N': 14.00307400443,
             'O': 15.99491461957,
             'F': 18.99840316273,
             'P': 30.97376199842,
             'S': 31.9720711744,
             'Cl': 34.968852682,
             'Ar': 39.9623831237,
             'K': 38.9637064864,
             'Ca': 39.962590863,
             'Si': 27.976926535,
             'Na':22.98976928
             }

In [88]:
def adduct_clean(string):
    if string == '[M + H]+':
        return '[M+H]+'
    elif string == '[M]+':
        return '[M]+'
    elif string == '[M - H2 + H]+':
        return '[M-H]+'
    else:
        return 'others'
    
def formula2dict(formula):
    lst = re.findall(r'[A-Z][a-z]*|\d+', re.sub('[A-Z][a-z]*(?![\da-z])', r'\g<0>1', formula))
    res_dct = {lst[i]: int(lst[i + 1]) for i in range(0, len(lst), 2)}
    return res_dct

def dict2formula(inputdict):
    formula = ''
    for key, value in inputdict.items():
        formula += key
        formula += str(value)
    return formula
    
    
def formula2dictH(formula):# this is because some bugs from sirius when dealing with [M]+
    lst = re.findall(r'[A-Z][a-z]*|\d+', re.sub('[A-Z][a-z]*(?![\da-z])', r'\g<0>1', formula))
    res_dct = {lst[i]: int(lst[i + 1]) for i in range(0, len(lst), 2)}
    res_dct['H'] = res_dct['H']+1
    return res_dct

def f2d(ionization, formula):
    if ionization != '[M]+':
        return formula2dict(formula)
    else:
        return formula2dictH(formula)
    
def loss(inputdict):
    inputdict['H'] = inputdict['H'] - 3
    try:
        inputdict['C'] = inputdict['C'] - 1
    except:
        pass
    return inputdict

def addloss(inputdict):
    inputdict['H'] = inputdict['H'] + 3
    try:
        inputdict['C'] = inputdict['C'] + 1
    except:
        pass
    return inputdict


def silylation(inputdict):
    try:
        si = inputdict['Si'] 
        inputdict['H'] = inputdict['H'] - 8*si
        inputdict['C'] = inputdict['C'] - 3*si
        inputdict.pop('Si')
    except:
        pass
    return inputdict
    
    
def calculate_mass(formula: str):
    #from yuanyue
    
    mol_mass = 0.
    try:
        all_atom_nums = re.findall('([A-Z][a-z]*)([0-9]*)', formula)
        for atom_num in all_atom_nums:
            n = atom_num[1]
            if n == '':
                mol_mass += atom_mass[atom_num[0]]
            else:
                mol_mass += int(n) * atom_mass[atom_num[0]]
    except KeyError as e:
        print("Atom {} is not known".format(e.args[0]))
    except TypeError:
        print('formula is nan')
    return round(mol_mass,6)

## Write down unknown spectra

In [21]:
index = [86, 361, 455, 92, 241, 253, 36, 182, 272, 359, 178, 145, 239, 135]

In [24]:
msp = './jack/CInofilter042822_Jake Pool sample_sudoM.mgf'

with open(msp, 'r') as f:
    tmp = f.readlines()
    f.close()
    
tmp2 = []    
i = 0
for spec in rdmsp(tmp):
    
    if i in index:
        
        tmp2 += spec
        tmp2 += ['\n']
    i += 1
    
with open('./jack/unknown.mgf','w') as f:
    f.writelines(tmp2)
        

## Read SIRIUS

In [54]:
sirius = './jack/unknown/'
sirius_15 = './jack/unknown_15loss/'

In [102]:
#both M and M - 15
rank_list = []
name = []
adductformula = []
path = sirius_15
ch3loss = False
for directory in os.listdir(path):

    
    if os.path.isdir(path+directory):
        
        print(directory)
        index = int(re.findall(r'\d+',directory)[0]) #-1266
        name.append(index)
        predict = pd.read_csv(path+directory+'/formula_candidates.tsv', sep='\t')

        predict['ionization'] = predict['adduct'].apply(adduct_clean)
        predict['dict']  = predict.apply(lambda x: f2d(x['ionization'], x['molecularFormula']), axis=1)
        
        predict['dict'] = predict['dict'].apply(addloss)
        predict['dict'] = predict['dict'].apply(silylation) 
        predict['formula'] = predict['dict'].apply(dict2formula)
        predict_15 = predict
        
        # ch3 loss
        path = sirius
        for directory in os.listdir(path):
            if os.path.isdir(path+directory):
                if index == int(re.findall(r'\d+',directory)[0]):
                    predict = pd.read_csv(path+directory+'/formula_candidates.tsv', sep='\t')

                predict['ionization'] = predict['adduct'].apply(adduct_clean)
                predict['dict']  = predict.apply(lambda x: f2d(x['ionization'], x['molecularFormula']), axis=1)

                predict['dict'] = predict['dict'].apply(silylation) 
                predict['formula'] = predict['dict'].apply(dict2formula)
        break

0_un_37


In [167]:
# only M
rank_list = []
name = []
adductformula = []
path = sirius
n_mol = []
mol = []
ch3loss = False
for directory in os.listdir(path):

    
    if os.path.isdir(path+directory):
        
        print(directory)
        index = int(re.findall(r'\d+',directory)[0]) #-1266
        name.append(index)    
        predict = pd.read_csv(path+directory+'/formula_candidates.tsv', sep='\t')
        predict['ionization'] = predict['adduct'].apply(adduct_clean)
        predict['dict']  = predict.apply(lambda x: f2d(x['ionization'], x['molecularFormula']), axis=1)
        
        predict['dict'] = predict['dict'].apply(silylation) 
        predict['formula'] = predict['dict'].apply(dict2formula)
        mol.append(predict['formula'][0])
        n_mol.append(getnmol(predict['formula'][0]))
        

0_unknown_1
10_unknown_11
11_unknown_12
12_unknown_13
13_unknown_14
1_unknown_2
2_unknown_3
3_unknown_4
4_unknown_5
5_unknown_6
6_unknown_7
7_unknown_8
8_unknown_9
9_unknown_10


In [168]:
df = pd.DataFrame({'name':name, 'formula':mol,'n_mol':n_mol})
df.to_csv('unknown_formula.csv')

In [158]:
import requests

#x = requests.get('https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/fastformula/C6OH12/cids/json')

x = requests.get('https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/fastformula/C15H28O5/property/InChI/json')


In [160]:
def getnmol(formula):
#formula = 'C6OH12'
    url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/fastformula/{formula}/property/InChI/json'
    x = requests.get(url)
    mol = x.json()['PropertyTable']['Properties']
    return len(mol)

def getmol(formula):
#formula = 'C6OH12'
    url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/fastformula/{formula}/property/InChI/json'
    x = requests.get(url)
    mol = x.json()['PropertyTable']['Properties']
    return mol



In [159]:
x.json()['PropertyTable']

{'Properties': [{'CID': 10469389,
   'InChI': 'InChI=1S/C15H28O5/c1-3-5-7-9-14(17)19-12-13(11-16)20-15(18)10-8-6-4-2/h13,16H,3-12H2,1-2H3/t13-/m0/s1'},
  {'CID': 152222,
   'InChI': 'InChI=1S/C15H28O5/c1-3-5-7-9-14(17)19-12-13(11-16)20-15(18)10-8-6-4-2/h13,16H,3-12H2,1-2H3'},
  {'CID': 131751198,
   'InChI': 'InChI=1S/C15H28O5/c16-13(10-7-8-12-15(19)20)9-5-3-1-2-4-6-11-14(17)18/h13,16H,1-12H2,(H,17,18)(H,19,20)'},
  {'CID': 86289229,
   'InChI': 'InChI=1S/C15H28O5/c1-3-5-7-9-14(17)19-12-13(11-16)20-15(18)10-8-6-4-2/h13,16H,3-12H2,1-2H3/t13-/m1/s1'},
  {'CID': 44370363,
   'InChI': 'InChI=1S/C15H28O5/c1-2-3-4-5-6-7-8-9-10-15(20,11-13(16)17)12-14(18)19/h20H,2-12H2,1H3,(H,16,17)(H,18,19)'},
  {'CID': 139586909,
   'InChI': 'InChI=1S/C15H28O5/c1-15(2,19)8-6-4-3-5-7-12(17)13-11(9-16)10-20-14(13)18/h11-13,16-17,19H,3-10H2,1-2H3/t11-,12?,13+/m1/s1'},
  {'CID': 254387,
   'InChI': 'InChI=1S/C15H28O5/c1-4-5-6-7-8-9-10-11-12-19-14(16)13(2)20-15(17)18-3/h13H,4-12H2,1-3H3'},
  {'CID': 115809,
   '

In [145]:
mol = x.json()['PropertyTable']['Properties']

In [131]:
len(cid)

734

In [155]:
mol

[{'CID': 7966, 'InChI': 'InChI=1S/C6H12O/c7-6-4-2-1-3-5-6/h6-7H,1-5H2'},
 {'CID': 5281167,
  'InChI': 'InChI=1S/C6H12O/c1-2-3-4-5-6-7/h3-4,7H,2,5-6H2,1H3/b4-3-'},
 {'CID': 7909, 'InChI': 'InChI=1S/C6H12O/c1-5(2)4-6(3)7/h5H,4H2,1-3H3'},
 {'CID': 6184, 'InChI': 'InChI=1S/C6H12O/c1-2-3-4-5-6-7/h6H,2-5H2,1H3'},
 {'CID': 5318042,
  'InChI': 'InChI=1S/C6H12O/c1-2-3-4-5-6-7/h4-5,7H,2-3,6H2,1H3/b5-4+'},
 {'CID': 11583, 'InChI': 'InChI=1S/C6H12O/c1-3-4-5-6(2)7/h3-5H2,1-2H3'},
 {'CID': 11509, 'InChI': 'InChI=1S/C6H12O/c1-3-5-6(7)4-2/h3-5H2,1-2H3'},
 {'CID': 31245, 'InChI': 'InChI=1S/C6H12O/c1-3-4-6(2)5-7/h5-6H,3-4H2,1-2H3'},
 {'CID': 20928, 'InChI': 'InChI=1S/C6H12O/c1-3-5-6(7)4-2/h4,6-7H,2-3,5H2,1H3'},
 {'CID': 7359, 'InChI': 'InChI=1S/C6H12O/c1-3-6(4-2)5-7/h5-6H,3-4H2,1-2H3'},
 {'CID': 129, 'InChI': 'InChI=1S/C6H12O/c1-6(2)4-3-5-7/h5-6H,3-4H2,1-2H3'},
 {'CID': 11262, 'InChI': 'InChI=1S/C6H12O/c1-4-5(2)6(3)7/h5H,4H2,1-3H3'},
 {'CID': 8108, 'InChI': 'InChI=1S/C6H12O/c1-3-5-6-7-4-2/h4H,2-3,5-6H2,

In [156]:
len(mol)

734